In [1]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load
from sklearn.metrics import classification_report

<Figure size 1000x600 with 0 Axes>

In [2]:
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000) #700 * 60 * 10 (10mins)

Using ready-made DataFrame


In [3]:
df.shape

(30, 59, 20)

In [4]:
y = df[:, 0, 0]  # 取得 'label'
X = df[:, :, 2:]  # 移除 'label','subject'

In [5]:
y=y-1 #1,2->0,1

In [6]:
X = np.array(X, dtype=np.float32)

y = np.array(y, dtype=np.int32)

print("X dtype:", X.dtype)  # 應該是 float32
print("y dtype:", y.dtype)  # 應該是 int32

X dtype: float32
y dtype: int32


In [7]:
# 分割訓練測試集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [42]:
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [43]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.LSTM(32, return_sequences=False))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [44]:
input_shape = (X_train.shape[1], X_train.shape[2]) #(59,18)
num_classes = len(np.unique(y_train))

print(input_shape)
print(num_classes)

(59, 18)
2


In [45]:
# 初始化模型
model = build_cnn_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 50
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/50


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6190 - loss: 0.6517 - val_accuracy: 0.4444 - val_loss: 2.2251
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5238 - loss: 1.8774 - val_accuracy: 0.4444 - val_loss: 0.9955
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5238 - loss: 0.8201 - val_accuracy: 0.5556 - val_loss: 0.7943
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5238 - loss: 0.8352 - val_accuracy: 0.5556 - val_loss: 0.8800
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5238 - loss: 0.9562 - val_accuracy: 0.5556 - val_loss: 0.7373
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5238 - loss: 0.7468 - val_accuracy: 0.6667 - val_loss: 0.6966
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5714 - loss: 0.6256 - val_accuracy: 0.4444 - val_loss: 0.8096
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5238 - loss: 0.6716 - val_accuracy: 0.4444 - val_loss: 0.8671
Epoch 9/50
1/

In [46]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
              precision    recall  f1-score   support

           1       0.75      0.60      0.67         5
           2       0.60      0.75      0.67         4

    accuracy                           0.67         9
   macro avg       0.68      0.68      0.67         9
weighted avg       0.68      0.67      0.67         9



In [47]:
# 初始化模型
model = build_cnn_lstm_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 50
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/50


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.4762 - loss: 0.6936 - val_accuracy: 0.4444 - val_loss: 0.6994
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5238 - loss: 0.6896 - val_accuracy: 0.4444 - val_loss: 0.6988
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5238 - loss: 0.6859 - val_accuracy: 0.4444 - val_loss: 0.6930
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5238 - loss: 0.6808 - val_accuracy: 0.4444 - val_loss: 0.6933
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5238 - loss: 0.6729 - val_accuracy: 0.7778 - val_loss: 0.6845
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6667 - loss: 0.6680 - val_accuracy: 0.4444 - val_loss: 0.6963
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5238 - loss: 0.6596 - val_accuracy: 0.5556 - val_loss: 0.6766
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7143 - loss: 0.6423 - val_accuracy: 0.7778 - val_loss: 0.6659
Epoch 9/50
1/

In [48]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
              precision    recall  f1-score   support

           1       1.00      0.80      0.89         5
           2       0.80      1.00      0.89         4

    accuracy                           0.89         9
   macro avg       0.90      0.90      0.89         9
weighted avg       0.91      0.89      0.89         9



In [49]:
# 選擇 LSTM 是因為 生理訊號屬於時間序列數據，LSTM 能夠記住過去的資訊，並利用這些資訊影響未來的預測。
# 此外，LSTM 透過 記憶單元與門機制，有效解決了傳統 RNN 中的 梯度消失問題，使模型能夠學習並捕捉長時間的依賴關係，從而提升壓力偵測的準確性。